In [1]:
!python3 -m pip install wget
import wget
!python3 -m wget https://github.com/mmanzato/6EABDARecSys/raw/main/dataset/AmazonMusic.tar.xz
!tar -xf AmazonMusic.tar.xz
!python3 -m pip install caserecommender

100% [....................................................] 23861964 / 23861964
Saved under AmazonMusic.tar.xz
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_json('./AmazonMusic/Digital_Music_5.json', lines=True)
dataset_metadata = pd.read_csv('./AmazonMusic/amazon_music_metadata.csv')

df = dataset[['reviewerID', 'asin', 'overall']]
df = df.merge(dataset_metadata[['asin', 'title']])

map_users = {user: idx for idx, user in enumerate(df.reviewerID.unique())}
map_items = {item: idx for idx, item in enumerate(df.asin.unique())}
df['reviewerID'] = df['reviewerID'].map(map_users)
df['asin'] = df['asin'].map(map_items)
    
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

train.to_csv('train.txt', index=False, header=False, sep='\t')
test.to_csv('test.txt', index=False, header=False, sep='\t')

In [5]:
dataset_metadata.head()

,asin,title,Accessories,Acid Jazz,Acoustic Blues,Adult Alternative,Adult Contemporary,Africa,Afro Brazilian,Afro-Cuban,...,Wall Stickers,Wall Switches,Washers,Wave Washers & Wave Springs,Wedding Music,West Coast,West Coast Blues,Western Swing,World Dance,World Music
0,5555991584,Memory of Trees,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6308051551,Dont Drink His Blood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7901622466,On Fire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B0000000ZW,Changing Faces,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B00000016W,Pet Sounds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
new_metadata = dataset_metadata.iloc[:,1:]
new_metadata = new_metadata.melt(id_vars=["title"])
new_metadata = new_metadata[new_metadata.value != 0]
new_metadata.reset_index(inplace=True, drop=True)
new_metadata.head()

,title,variable,value
0,Light to Dark,Acid Jazz,1.0
1,Supernatural,Acid Jazz,1.0
2,Party to the Bus Stop,Acid Jazz,1.0
3,Stratosphere Breakdown,Acid Jazz,1.0
4,Mission Impossible,Acid Jazz,1.0


In [9]:
map_title = {}
inverse_map_title = {}

for _, row in df.iterrows():
    map_title[row.asin] = row.title
    inverse_map_title[row.title] = row.asin

In [10]:
new_metadata['asin_id'] = new_metadata['title'].map(inverse_map_title)

In [11]:
new_metadata.dropna(inplace=True)
new_metadata = new_metadata[['asin_id', 'variable', 'value']]
new_metadata['asin_id'] = new_metadata.asin_id.astype(int)
new_metadata

,asin_id,variable,value
6,132,Acid Jazz,1.0
8,243,Acid Jazz,1.0
11,545,Acid Jazz,1.0
12,587,Acid Jazz,1.0
13,601,Acid Jazz,1.0
...,...,...,...
62553,2541,World Music,1.0
62561,2550,World Music,1.0
62562,2555,World Music,1.0
62563,2555,World Music,1.0


In [12]:
new_metadata.to_csv('items_metadata.dat', index=False, sep='\t', header=False)

### Case Recommender


In [13]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

In [15]:
ItemAttributeKNN('train.txt', 'test.txt', metadata_file='items_metadata.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 5099 users and 2582 items (41436 interactions) | sparsity:: 99.69%
test data:: 3818 users and 2288 items (10360 interactions) | sparsity:: 99.88%

training_time:: 5.963499 sec
>> metadata:: 2521 items and 292 metadata (26048 interactions) | sparsity:: 96.46%
prediction_time:: 0.506548 sec
Eval:: MAE: 0.689393 RMSE: 0.976026 
